# Comparison for LogSumExp

The LogSumExp-problem with linear consraints:
	$$\log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2 \rightarrow \min_{\textbf{x}\in \mathbb{R}^N}$$
	$$\text{s.t.} (\textbf{b}_j, \textbf{x}) - c_j <= 0, j = 1\dots M$$
	
This problem can be converted into saddle point problem:
	$$\min_{\textbf{x}} \max_{\textbf{y}\in\mathbb{R}^M_+} S(\textbf{x},\textbf{y}),$$
where $S(\textbf{x}, \textbf{y}) = r(\textbf{x})+F(\textbf{x}, \textbf{y}) - h(\textbf{y})$
	$$r(\textbf{x}) = \log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2$$
	$$F(\textbf{x}, \textbf{y}) = \sum\limits_{j=1}^M x_j(\textbf{b}_j, \textbf{y}) = (\textbf{x}, B\textbf{y})$$
	$$h(\textbf{y}) = (\textbf{c}, \textbf{y})$$

So N is dimensional of external variable, M - internal. But we will consider the external problem as help problem.

In [1]:
import numpy as np
from TestFunctions import LogSumExp_difalpha as LogSumExp
from TestFunctions.TestFunctions import TestFunction
from Solvers.GradientMethods import FGM_internal
import matplotlib.pyplot as plt
%matplotlib notebook
import comparison
import imp

from comparison import method_comparison, create_methods_dict

In [2]:
def get_LSE(N = 100, M = 2, alpha = 0.001, beta = 0.5e-3, k = 1000, seed = 241):
    """
    The function is to get class of LSE. All parameters mathes the
    parameters above.
    
    N is dimensional of primal problem
    M is number of constraints. It is also the dual dimension
    alpha is parameter of LogSumExp problem
    beta is coefficient of l2-regularization for primal problem
    
    k is parameter for to generate matrix B
    """
    np.random.seed(M*N)
    alpha_ = np.random.uniform(size = (N,), low = -alpha, high = alpha)
    alpha = np.max(np.abs(alpha))
    B = []
    for i in range(M):
        b = np.random.uniform(low = -k, high = 0, size = (N,)) * np.sign(alpha_)
        B.append(b)
    B = np.array(B)
    #B = np.random.uniform(low = -k, high = 0, size = (M, N))
    c = np.ones((M,))
    x_ = np.zeros((N,))
    f_ = LogSumExp.r(alpha, beta).get_value(x_)
    gamma = c.max()
    y_max = f_/gamma
    Q = [[0, y_max] for i in range(M)]
    Q = np.array(Q)
    size_domain = np.linalg.norm(Q[:,0]-Q[:,1])
    LSE = TestFunction(r = LogSumExp.h(c, size_domain = size_domain),
                    F = LogSumExp.F_tilde(B, size_domain),
                    h = LogSumExp.r(alpha_, beta, size_domain),
                    solver = FGM_internal, 
                    get_start_point = lambda x: (1/beta * -x.dot(B), alpha * np.sqrt(N)/beta))
    return LSE, Q, alpha_, B, c, beta

In [3]:
def parse_results(history, alpha_, B, c, beta, keys = ["Ellipsoids", "Dichotomy", "FGM"]):
    s_key = ""
    s_feas = ""
    s_time = ""
    s_value = ""
    for key in keys:
        s_key += key + "\t$"
        lambda_, x = history[key][-1][0]        
        s_feas += str((B @ x <= c).any()) + "\t\t$"
        s_time += "%.3f"%(history[key][-1][1] - history[key][0][1]) + "\t\t$"
        s_value+= "%.3f"%(np.log(1 + np.exp(alpha_*x).sum()) + beta * np.linalg.norm(x)**2) + "\t\t$"
    print(s_key)
    print(s_feas)
    print(s_time)
    print(s_value)

## M= 2

In [4]:
M = 2

### N = 100

In [5]:
N = 100

In [6]:
eps = 1e-3

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/GradientMethods.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  est = 2 * min(4 * L * R/N**2, L * R * np.exp(-N/2 * np.sqrt(mu/L)))


In [7]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
0.392		$0.083		$20.006		$
4.615		$4.615		$4.615		$


In [8]:
eps = 1e-6

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [9]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
1.016		$0.249		$20.002		$
4.615		$4.615		$4.615		$


In [10]:
eps = 1e-9

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [11]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
2.599		$1.561		$20.010		$
4.615		$4.615		$4.615		$


### N = 1000

In [12]:
N = 1000

In [13]:
eps = 1e-3

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [14]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
0.589		$0.150		$20.010		$
6.910		$6.910		$6.909		$


In [15]:
eps = 1e-6

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [16]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
2.994		$1.485		$20.015		$
6.909		$6.909		$6.909		$


In [17]:
eps = 1e-9

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [18]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
7.237		$7.227		$20.019		$
6.909		$6.909		$6.909		$


## M= 3

In [19]:
M = 3

### N = 100

In [20]:
N = 100

In [21]:
eps = 1e-3

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [22]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
0.482		$0.184		$20.003		$
4.615		$21.622		$4.615		$


In [23]:
eps = 1e-6

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [24]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
2.849		$1.484		$20.004		$
4.615		$4.615		$4.615		$


In [25]:
eps = 1e-9

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [26]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
9.750		$12.410		$20.002		$
4.615		$4.615		$4.615		$


### N = 1000

In [27]:
N = 1000

In [28]:
eps = 1e-3

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [29]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
1.950		$0.693		$20.000		$
6.909		$6.910		$6.909		$


In [30]:
eps = 1e-6

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [31]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
14.475		$14.250		$20.003		$
6.909		$6.909		$6.909		$


In [32]:
eps = 1e-9

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 40,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [33]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
19.689		$30.940		$40.000		$
6.909		$6.909		$6.909		$


## M= 4

In [34]:
M = 4

### N = 100

In [35]:
N = 100

In [36]:
eps = 1e-3

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [37]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
1.715		$0.435		$20.002		$
4.615		$4.615		$4.615		$


In [38]:
eps = 1e-6

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [39]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
4.575		$2.541		$20.003		$
4.615		$4.615		$4.615		$


In [40]:
eps = 1e-9

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [41]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
15.209		$20.316		$20.005		$
4.615		$4.615		$4.615		$


### N = 1000

In [42]:
N = 1000

In [43]:
eps = 1e-3

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [44]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
5.729		$5.481		$20.005		$
6.909		$6.909		$6.909		$


In [45]:
eps = 1e-6

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 20,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [46]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
19.056		$20.158		$20.023		$
6.909		$6.909		$6.909		$


In [51]:
eps = 1e-9

LSE, Q, alpha_, B, c, beta = get_LSE(N, M)
history = {}
methods = create_methods_dict(LSE, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = 300,
                             stop_cond_args = [alpha_, B, c, beta])
comparison.method_comparison(methods)

In [52]:
parse_results(history, alpha_, B, c, beta)

Ellipsoids	$Dichotomy	$FGM	$
True		$True		$False		$
32.583		$88.841		$300.001		$
6.909		$6.909		$6.909		$
